### 0.引用相应的包

In [1]:
import pandas as pd  
import numpy as np
import datetime
import backtrader as bt
import matplotlib.pyplot as plt
%matplotlib inline   

#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

### 1.数据准备（Data）

In [2]:
datapath=r'D:\Baiducloud\12M8-202005\1MIN\EURUSD_M1.csv'
dt=pd.read_csv(datapath)

In [4]:
dt.rename(columns={'Time (UTC)':'Date'},inplace=True)
dt.index=pd.to_datetime(dt.Date)

In [5]:
dt.drop(columns='Date')

,Open,High,Low,Close,Volume
Date,,,,,
2010-01-01 00:00:00,1.43283,1.43283,1.43267,1.43267,74.3
2010-01-01 00:01:00,1.43268,1.43285,1.43268,1.43284,25.4
2010-01-01 00:02:00,1.43280,1.43280,1.43268,1.43280,30.9
2010-01-01 00:03:00,1.43280,1.43280,1.43268,1.43273,41.9
2010-01-01 00:04:00,1.43283,1.43283,1.43277,1.43277,53.8
...,...,...,...,...,...
2020-05-01 20:55:00,1.09513,1.09513,1.09513,1.09513,0.0
2020-05-01 20:56:00,1.09513,1.09513,1.09513,1.09513,0.0
2020-05-01 20:57:00,1.09513,1.09513,1.09513,1.09513,0.0


In [12]:
import datetime as datetime
data = bt.feeds.PandasData(dataname=dt,                               
                            fromdate=datetime.datetime(2019, 10, 1,0,0,0),                               
                            todate=datetime.datetime(2020, 4, 30,0,0,0) )           #这里面要尤其注意

## 2.策略模块（Strategy） 

In [13]:
class MyStrategy(bt.Strategy):
    params=(('short',30),                   # RSI的下限
            ('long',70),)                   # RSI的上限

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def next(self):
        if not self.position:
            if self.rsi < self.params.short:
                self.buy()
        else:
            if self.rsi > self.params.long:
                self.sell()

In [ ]:
#   if __name__ == '__main__':   #在py文件里面运行需要
    
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(MyStrategy)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(1000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Run over everything
    cerebro.run()